In [4]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [5]:
!which chromedriver

/usr/local/bin/chromedriver


In [6]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
#NASA Mars News
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(5)
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
mars = soup.find('ul', class_='item_list').find('li', class_='slide')
news_title = mars.find("div", class_="content_title").get_text()
print(news_title)

MOXIE Could Help Future Rockets Launch Off Mars


In [12]:
news_body = mars.find("div", class_= "article_teaser_body").get_text()
print(news_body)

NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [13]:
#JPL Mars Space Images - Featured Image
url_jpl = "https://www.jpl.nasa.gov"
url_jpl_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_jpl_image)
time.sleep(5)
browser.click_link_by_partial_text('FULL IMAGE')
html = browser.html
soup = bs(html, 'html.parser')
image = soup.find("img", class_="fancybox-image")["src"]
featured = f'{url_jpl}{image}'
print(featured)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18328_ip.jpg


In [14]:
#Mars Facts
mars = "https://space-facts.com/mars/"
mars_table = pd.read_html(mars)
mars_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [15]:
mars_df = mars_table[0]
mars_df = mars_df.rename(columns={0: "Description", 1: "Mars"}, errors="raise")
mars_df.set_index("Description", inplace=True)
#mars_df = mars_df.rename_axis(None)
mars_df


,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
#convert the table to html
mars_html_table = mars_df.to_html()
mars_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

In [17]:
#Mars Hemispheres
#Scrape Mars hemisphere title and image
astro_url = "https://astrogeology.usgs.gov/"
hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_url)
html = browser.html
soup = bs(html, "html.parser")

In [19]:
#Extract Item elements
mars_hems = soup.find("div", class_="collapsible results")
mars_item = mars_hems.find_all("div", class_="item")
image_urls = []

In [22]:
#looping through hemisphere item
for item in mars_item:
    # Error handling
    try:
        #Extract title
        hem = item.find("div", class_="description")
        hem_title = hem.h3.text
        #Image url
        hem_image_url = hem.a["href"]
        browser.visit(astro_url+hem_image_url)
        html = browser.html
        soup = bs(html, "html.parser")
        image_src = soup.find('li').a['href']
        if (hem_title and image_src):
            print("-----------------------------")
            print(hem_title)
            print(image_src)
        hem_dict = {
            "Title" : hem_title,
            "Image Url" : image_src}
            
        image_urls.append(hem_dict)
    except Exception as e:
        print(e)



-----------------------------
Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
-----------------------------
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
-----------------------------
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
-----------------------------
Valles Marineris Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
